In [ ]:
df = spark.sql("select * from titanic_csv")     # a table

In [ ]:
my_cols = df.select['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']

In [ ]:
my_final_data = my_cols.na.drop()

In [2]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer

In [ ]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
# A B C
# 0 1 2
# ONE HOT ENCODE
# KEY A B C
# Example A
# [1,0,0]
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')


In [ ]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [ ]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],outputCol='features')

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [ ]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [ ]:
pipeline = Pipeline(stages = [gender_indexer,embark_indexer,gender_encoder,embark_encoder,assembler,log_reg_titanic])

In [ ]:
train_data,test_data = my_final_data.randomSplit([0.7,0.3])

In [ ]:
fit_model = pipeline.fit(train_data)

In [ ]:
results = fit_model.transform(test_data)

In [3]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')
my_eval.evaluate(results)